오늘은, 노래가사를 input data로 이용하여, 작사를 하는 AI인공지능을 만들어 보았습니다.

제가 사용한 노래는 Adele, Beatles, Bieber, Britney-spears, Bruno Mars, Disney,michael-jackson, rihanna의 노래입니다.

제공받은 노래의 종류는 더 많지만 그것을 전부 사용하지 않은 이유는 다음과 같습니다.

1. 전체 노래를 학습시키기에는 양이 너무 많다. 학습시킬 데이터 양이 많으면 시간이 매우 오래걸리기 때문에 선별이 필요했습니다.
2. 전처리를 위해서는 데이터가 어떤 형식인지 대강이라도 먼저 살펴보아야 하는데, 그 모든 노래를 살펴볼수 없었다.
3. 제가 아는 가수 위주로 골랐습니다.
    - 이 이유는, 노래의 분위기를 맞춰주려고 한 것입니다. 예를 들어 신문기사를 쓰는 AI를 작성하려는데, 아동의 동요 데이터도 같이 학습 시킨다면, 이는 노이즈 데이터라고 생각합니다.
    - 이와 같은 이유로, eminem가수를 저는 알지만, disney노래와는 잘 맞지 않는 분위기입니다. 그래서 제외시켰습니다.

In [1]:
import glob
import os
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split

txt_file_path = os.getenv('HOME')+'/aiffel/Aiffel_Exploration_nodes/lyricist/mydata/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 32649
Examples:
 ['Looking for some education', 'Made my way into the night', 'All that bullshit conversation']


# 1. 데이터 전처리하기

제가 전처리로 삭제시킨 데이터의 리스트는 다음과 같습니다.

1. Nah nah nah nah nah nah, nah nah nah, hey Jude와 같이 여러 줄 / 추임새 단어가 반복되는 문장은 제외시켰습니다.
    - 이 모델의 목표는 노래 전부를 작사하는 것이 아니라, 가사 한 줄을 생성하는 것입니다. 만약, 노래 전부를 작사하는 것이라면 oh / Nah / yeah와 같은 감탄사 혹은 추임새가 노래에서 중요한 부분을 차지합니다. 하지만 한 줄의 가사를 생성하는데, 추임새와 감탄사만 나오는 가사가 생성이 된다면, (물론 의미는 있겠지만) 제가 원하는 방향이 아닙니다.
2. 괄호 안의 문장은 삭제했습니다.    
    - 1줄 작사에 괄호가 필요하지 않을 뿐더러, 그저 같은 가사 반복인 경우가 많음
3. 5번 이상 반복되는 같은 문장
    - 영향을 많이 받을것 같아 삭제하였습니다.
4. 각종 노래 지시사항
    - [x2:]
    - 
6. 문장의 길이가 5이하인 문장
    - 추임새로만 이루어진 문장

In [2]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#        strip문자의 특성
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


삭제 리스트 
- 문장 자체


- 이거만 으로 구성된 문장


- 이거 포함된 문장






삭제 고민중 :

In [3]:
# 여기에 정제된 문장을 모을겁니다
corpus = []


for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.
    if sentence[-1] == "]": continue # ']'로 끝나는 파트를 나눈 문장은 건너뜁니다
    if sentence[-1] == ")": continue # ')'로 끝나는 코러스 문장은 건너뜁니다    

    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
corpus[:10]
    

['<start> looking for some education <end>',
 '<start> made my way into the night <end>',
 '<start> all that bullshit conversation <end>',
 '<start> baby , can t you read the signs ? i won t bore you with the details , baby <end>',
 '<start> i don t even wanna waste your time <end>',
 '<start> let s just say that maybe <end>',
 '<start> you could help me ease my mind <end>',
 '<start> i ain t mr . right but if you re looking for fast love <end>',
 '<start> if that s love in your eyes <end>',
 '<start> it s more than enough <end>']

In [4]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=10000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2 209  31 ...   0   0   0]
 [  2 266  14 ...   0   0   0]
 [  2  25  17 ...   0   0   0]
 ...
 [  2  22  22 ...   0   0   0]
 [  2   6  19 ...   0   0   0]
 [  2  22  22 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f697e0179a0>


In [5]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 50: break
#어떻게 구축되었는지.        

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : you
6 : i
7 : the
8 : me
9 : it
10 : to
11 : and
12 : t
13 : a
14 : my
15 : s
16 : in
17 : that
18 : love
19 : m
20 : your
21 : we
22 : oh
23 : be
24 : don
25 : all
26 : .
27 : on
28 : can
29 : of
30 : is
31 : for
32 : like
33 : re
34 : so
35 : know
36 : baby
37 : just
38 : yeah
39 : what
40 : but
41 : no
42 : do
43 : ?
44 : with
45 : got
46 : she
47 : when
48 : let
49 : ll
50 : up


In [6]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    
print(len(src_input))
print(src_input[9])
print(tgt_input[9])

28307
[  2   9  15  98 197 190   3   0   0   0   0   0   0   0]
[  9  15  98 197 190   3   0   0   0   0   0   0   0   0]


In [7]:
# 평가 데이터셋 분리

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                    tgt_input, 
                                                    test_size=0.2)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (22645, 14)
Target Train: (22645, 14)


In [8]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE
print(BUFFER_SIZE, steps_per_epoch)

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

28307 110


<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [9]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [10]:
    
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1):break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 14, 10001), dtype=float32, numpy=
array([[[ 2.8559431e-05, -2.5264348e-04,  1.3473186e-04, ...,
         -3.1647796e-04,  4.1535153e-05,  2.2057720e-04],
        [-1.7098541e-04, -4.2430044e-04,  4.9477174e-05, ...,
         -3.9836805e-04, -8.7433778e-05,  6.3471479e-04],
        [-9.9618141e-05, -4.8712018e-04, -7.9317600e-05, ...,
         -3.0250222e-04,  7.7358964e-05,  6.9642020e-04],
        ...,
        [-1.1163973e-03,  1.8952154e-04,  7.7737449e-04, ...,
          8.0995489e-04,  5.0952687e-04,  9.2478713e-04],
        [-1.5072697e-03,  1.8810506e-04,  8.2107796e-04, ...,
          9.3263719e-04,  5.6689518e-04,  1.0447112e-03],
        [-1.9069926e-03,  2.0549900e-04,  8.1067678e-04, ...,
          1.0044173e-03,  6.1418233e-04,  1.1957278e-03]],

       [[ 2.8559431e-05, -2.5264348e-04,  1.3473186e-04, ...,
         -3.1647796e-04,  4.1535153e-05,  2.2057720e-04],
        [ 3.7472484e-05, -3.1124725e-04,  3.6284511e-04, ...,
         -5.3073576e-04, 

In [11]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  2560256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  10251025  
Total params: 26,450,961
Trainable params: 26,450,961
Non-trainable params: 0
_________________________________________________________________


In [12]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
110/110 [==============================] - 18s 146ms/step - loss: 3.9994
Epoch 2/30
110/110 [==============================] - 17s 151ms/step - loss: 3.2265
Epoch 3/30
110/110 [==============================] - 17s 155ms/step - loss: 3.0248
Epoch 4/30
110/110 [==============================] - 17s 151ms/step - loss: 2.8605
Epoch 5/30
110/110 [==============================] - 16s 149ms/step - loss: 2.7395
Epoch 6/30
110/110 [==============================] - 16s 149ms/step - loss: 2.6387
Epoch 7/30
110/110 [==============================] - 17s 151ms/step - loss: 2.5496
Epoch 8/30
110/110 [==============================] - 17s 152ms/step - loss: 2.4658
Epoch 9/30
110/110 [==============================] - 17s 151ms/step - loss: 2.3855
Epoch 10/30
110/110 [==============================] - 17s 150ms/step - loss: 2.3098
Epoch 11/30
110/110 [==============================] - 17s 150ms/step - loss: 2.2374
Epoch 12/30
110/110 [==============================] - 17s 151ms/step - lo

In [13]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [14]:
generate_text(model, tokenizer, init_sentence="<start> he")

'<start> he s a monster <end> '